In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers
!pip install torch
!pip install PIL
!pip install accelerate
!pip install transformers

In [5]:
from PIL import Image

def image_grid(imgs, rows, cols, resize=256):
    assert len(imgs) == rows * cols

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]

    w, h = imgs[0].size
    grid_w, grid_h = cols * w, rows * h
    grid = Image.new("RGB", size=(grid_w, grid_h))

    for i, img in enumerate(imgs):
        x = i % cols * w
        y = i // cols * h
        grid.paste(img, box=(x, y))

    return grid

In [ ]:
!pip install -U autotrain-advanced

In [9]:
import os
os.environ['MODEL_NAME'] = f'stabilityai/stable-diffusion-xl-base-1.0'
os.environ['DATA_DIR'] = f'./data/louis_pics'
os.environ['PROJECT_NAME'] = f'Dreambooth_SDXL_MG'

In [ ]:
import torch
torch.cuda.empty_cache()
!nvidia-smi


In [ ]:

!autotrain dreambooth \
--model $MODEL_NAME \
--project-name $PROJECT_NAME \
--image-path $DATA_DIR \
--prompt "A photo of Louis." \
--resolution 512 \
--batch-size 1 \
--num-steps 500 \
--gradient-accumulation 4 \
--lr 1e-4 \
--fp16

In [ ]:
from diffusers import DiffusionPipeline, AutoencoderKL
import torch

vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", 
    torch_dtype=torch.float16
)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
pipe.to("cuda")

pipe.load_lora_weights("Dreambooth_SDXL_MG/pytorch_lora_weights.safetensors")
prompt = "portrait of mantagen, sun shining, photo realistic illustration by greg rutkowski, thomas kindkade, alphonse mucha, loish, norman rockwell."
image = pipe(prompt=prompt, num_inference_steps=24, num_images_per_prompt=3)
image_grid(image.images, 1, 3)
